In [ ]:
#nifit to dicom

import numpy as np
import pydicom
from pydicom.uid import generate_uid
from datetime import datetime
import SimpleITK as sitk
import os

def save_heatmaps_to_dicom(heatmap_volume, output_path, patient_name="HeatmapPatient", 
                           patient_id="HM001", series_description="Heatmaps", modality="OT"):
    if len(heatmap_volume.shape) != 3:
        raise ValueError(f"Expected heatmap volume to have 3 dimensions, but got {heatmap_volume.shape}")

    # dummy data
    ds = pydicom.Dataset()
    ds.file_meta = pydicom.Dataset()
    ds.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.PatientName = patient_name
    ds.PatientID = patient_id
    ds.Modality = modality
    ds.SeriesDescription = series_description
    ds.SeriesInstanceUID = generate_uid()
    ds.StudyInstanceUID = generate_uid()
    ds.SOPInstanceUID = generate_uid()
    ds.StudyDate = datetime.now().strftime("%Y%m%d")
    ds.StudyTime = datetime.now().strftime("%H%M%S")
    ds.ContentDate = ds.StudyDate
    ds.ContentTime = ds.StudyTime
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.Rows, ds.Columns = heatmap_volume.shape[1], heatmap_volume.shape[2]
    ds.NumberOfFrames = heatmap_volume.shape[0]
    ds.PixelRepresentation = 0  # Unsigned integer
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15

    # Scale heatmap data to fit within 16-bit unsigned integer range
    heatmap_volume = (heatmap_volume * 65535).astype(np.uint16)

    # Convert the heatmap volume to a byte array and store it in PixelData
    ds.PixelData = heatmap_volume.tobytes()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    pydicom.dcmwrite(output_path, ds)

    print(f"DICOM file saved at: {output_path}")
nifti_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_1_left_heatmaps.nii.gz"
heatmap_volume = sitk.GetArrayFromImage(sitk.ReadImage(nifti_path))  # Assuming the file exists and is a 64x64x64 
output_dicom_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/heatmap_volume.dcm"
save_heatmaps_to_dicom(heatmap_volume, output_dicom_path)
